In [1]:
testing = False;
parametroFichas = 0.11;
entrada = "instance";
version = "C";
solucion_inicial = True;
print(f"Configurado con testing = {testing}. Versión: {version}. Entrada: {entrada}. Solución inicial: {solucion_inicial}.");

Configurado con testing = False. Versión: C. Entrada: instance. Solución inicial: True.


In [ ]:
import warnings
import json
import pandas as pd
import random
import numpy as np
import time

warnings.filterwarnings("ignore");

if testing == False:
    with open(f"../input/{entrada}.json") as file:
        data = json.load(file);
else:
    with open("../input/inst_test.json") as file:
        data = json.load(file);
    
config = data["configurations"];
dfSolucion = pd.DataFrame();
dfSolucion.to_excel(f"../output/LPM/{entrada}_output.xlsx", index=False);
                    
dfSurgeon = pd.read_excel("../input/MAIN_SURGEONS.xlsx", sheet_name='surgeon', converters={'n°':int}, index_col=[0]);
dfSecond = pd.read_excel("../input/SECOND_SURGEONS.xlsx", sheet_name='second', converters={'n°':int}, index_col=[0]);
dfRoom = pd.read_excel("../input/ROOMS.xlsx", sheet_name='room', converters={'n°':int}, index_col=[0]);
dfType = pd.read_excel("../input/PROCESS_TYPE.xls", sheet_name='Process Type', converters={'n°':int}, index_col=[0]);
                    
extras = [];
dfdisORA = pd.read_excel("../input/DIST_OR_EXT.xlsx", sheet_name='A', converters={'n°':int}, index_col=[0]);
dfdisORA = dfdisORA.astype(str).values.tolist();
extraA = [];
for i in range(len(dfdisORA)):
    extraA.append(dfdisORA[i].copy());
for i in range(len(dfdisORA[0])):
    aux = dfdisORA[0][i].split(";");
    dfdisORA[0][i] = aux[0];
    extraA[0][i] = aux[1:];
    aux = dfdisORA[1][i];
    aux = aux.split(";");
    dfdisORA[1][i] = aux[0];
    extraA[1][i] = aux[1:];
    num_ext = len(extraA[1][i]);
extras.append(extraA);

dfdisORB = pd.read_excel("../input/DIST_OR_EXT.xlsx", sheet_name='B', converters={'n°':int}, index_col=[0]);
dfdisORB = dfdisORB.astype(str).values.tolist();
extraB = [];
for i in range(len(dfdisORB)):
    aux = dfdisORB[i].copy();
    extraB.append(aux);
for i in range(len(dfdisORB[0])):
    aux = dfdisORB[0][i];
    aux = aux.split(";");
    dfdisORB[0][i] = aux[0];
    extraB[0][i] = aux[1:];
    aux = dfdisORB[1][i];
    aux = aux.split(";");
    dfdisORB[1][i] = aux[0];
    extraB[1][i] = aux[1:];
extras.append(extraB);

dfdisORC = pd.read_excel("../input/DIST_OR_EXT.xlsx", sheet_name='C', converters={'n°':int}, index_col=[0]);
dfdisORC = dfdisORC.astype(str).values.tolist();
extraC = [];
for i in range(len(dfdisORC)):
    aux = dfdisORC[i].copy();
    extraC.append(aux);
for i in range(len(dfdisORC[0])):
    aux = dfdisORC[0][i];
    aux = aux.split(";");
    dfdisORC[0][i] = aux[0];
    extraC[0][i] = aux[1:];
    aux = dfdisORC[1][i];
    aux = aux.split(";");
    dfdisORC[1][i] = aux[0];
    extraC[1][i] = aux[1:];
extras.append(extraC);

dfdisORD = pd.read_excel("../input/DIST_OR_EXT.xlsx", sheet_name='D', converters={'n°':int}, index_col=[0]);
dfdisORD = dfdisORD.astype(str).values.tolist();
extraD = [];
for i in range(len(dfdisORD)):
    aux = dfdisORD[i].copy();
    extraD.append(aux);
for i in range(len(dfdisORD[0])):
    aux = dfdisORD[0][i];
    aux = aux.split(";");
    dfdisORD[0][i] = aux[0];
    extraD[0][i] = aux[1:];
    aux = dfdisORD[1][i];
    aux = aux.split(";");
    dfdisORD[1][i] = aux[0];
    extraD[1][i] = aux[1:];
extras.append(extraD);

dfdisAffi = pd.read_excel("../input/AFFINITY_EXT.xlsx", sheet_name='Hoja1', converters={'n°':int}, index_col=[0]);
dfdisAffiDiario = pd.read_excel("../input/AFFINITY_DIARIO.xlsx", sheet_name='Dias', converters={'n°':int}, index_col=[0]);
dfdisAffiBloque = pd.read_excel("../input/AFFINITY_DIARIO.xlsx", sheet_name='Bloques', converters={'n°':int}, index_col=[0]);
dfdisRank = pd.read_excel("../input/RANKING.xlsx", sheet_name = 'Hoja1', converters={'n°':int}, index_col=[0]);

extra = [];
dfExtra = [];
for i in range(num_ext):
    aux = pd.read_excel("../input/AFFINITY_EXT.xlsx", sheet_name='Extra'+str(i+1), converters = {'n°':int}, index_col=[0]);
    extra.append(len(aux));
    dfExtra.append(aux);

# Rankings para extras
dfRankExtra = [];
for i in range(num_ext):
    aux = pd.read_excel("../input/RANKING.xlsx", sheet_name='Extra'+str(i+1), converters = {'n°':int}, index_col=[0]);
    extra.append(len(aux));
    dfRankExtra.append(aux);

#-------------------------------------------------------------------------------------------------------------------------------#     
########################################################### MAIN LOOP ###########################################################
#-------------------------------------------------------------------------------------------------------------------------------#  

for INS in config:
    dfSolucion = pd.read_excel(f"../output/LPM/{entrada}_output.xlsx");
    #version = INS["version"];
    version = version;
    typePatients = INS["patients"];
    nPatients = int(INS["n_patients"]);
    nDays = int(INS["days"]);
    min_affinity = int(INS["min_affinity"]);
    nSurgeons = int(INS["surgeons"]);
    nFichas = int(INS["fichas"]);
    time_limit = int(INS["time_limit"]);

    patient_code = "0" if typePatients == "low" else ("1" if typePatients == "high" else "2");
    print(f"Versión: {version};\nPacientes: {typePatients}; Número de Pacientes: {nPatients};\nDías: {nDays};\nAfinidad: {min_affinity}; Cirujanos: {nSurgeons};\nFichas: {nFichas}.");
    
    #list_sol = [instancia[INS][0]+"_"+instancia[INS][1],"","","","","","","","","","","","","",""];
    dict_sol = {"Instancia": [f"v{version}p{patient_code}n{nPatients}s{nSurgeons}d{nDays}"]};
    
    if typePatients == "low":
        dfPatient = pd.read_csv("../input/LowPriority.csv");
    elif typePatients == "high":
        dfPatient = pd.read_csv("../input/HighPriority.csv");
    else:
        dfPatient = pd.read_csv("../input/AllPriority.csv");
    
    dfPatient = dfPatient.iloc[:nPatients];

    # Indices
    random.seed(0);
    patient = [p for p in range(nPatients)];
    surgeon = [s for s in range(nSurgeons)];
    second = [a for a in range(nSurgeons)]
    room = [o for o in range(len(dfRoom))]
    day = [d for d in range(nDays)];
    nSlot = 16;  # Bloques de 30 minutos
    nRooms = len(room);
    slot = [t for t in range(nSlot)]
    T = nSlot//2; # División entre mañana y tarde
    
    process=[t for t in range(len(dfType))] # ?
    level_affinity = min_affinity;

    # Arcos
    arcox = [(p,o,s,a,t,d) for p in patient for o in room for s in surgeon for a in second for t in slot for d in day];
    arcoy = [(p,o,d) for p in patient for o in room for d in day];
    arcoz = [(p,s,a) for p in patient for s in surgeon for a in second];
    arcot = [p for p in patient];
    arcof = [(s,d) for s in surgeon for d in [a for a in range(-1, nDays)]];

    #Max surgeries and budget per surgeon
    M = np.zeros(nSurgeons, dtype=int);
    Pr = np.zeros(nSurgeons, dtype=int);
    for s in surgeon:
        M[s] = int(dfSurgeon.iloc[s][8]); # Máximo de cirugías
        Pr[s] = int(dfSurgeon.iloc[s][11]); # Presupuesto de fichas

    E = np.ones((nSlot, nDays)) * 1000; # ? 
    A = np.ones((nSlot, nDays)) * 1000; # ?
    B = np.ones(nPatients) * 1; # ?
    Y = np.ones(nPatients) * 1; # ?

    # Prioridades
    I = np.ones((nPatients, nDays), dtype=int);
    for p in patient:
        for d in day:
            try:
                I[(p,d)] = 1 + dfPatient.iloc[p]["espera"] * dfPatient.iloc[p]["edad"] * 0.0001/(d+1);
            except ValueError:
                print("Value Error en cálculo de prioridades.");

    # Matriz de coincidencia cirujanos
    COIN = np.zeros((nSurgeons, nSurgeons), dtype=int);
    for s in surgeon:
        for a in second:
            if dfSurgeon.iloc[s][0] == dfSecond.iloc[a][0]:
                COIN[(s,a)] = 1;
            else:
                COIN[(s,a)] = 0;
                

    Ex = [np.ones((nSurgeons, extra[i]), dtype=float) for i in range(num_ext)];
    for i in range(num_ext):
        for s in surgeon:
            for e in range(extra[i]):
                Ex[i][(s,e)] = dfExtra[i].iloc[e][s+1];

    # Disponibilidad de cirujanos
    # SD = np.zeros((nSurgeons, nSurgeons, nSlot, nDays%5), dtype=int);
    # for s in surgeon:
    #     for a in second:
    #         for d in day:
    #             if dfSurgeon.iloc[s][2+d%5] == 1:
    #                 if dfSecond.iloc[a][2+d%5] == 1:
    #                     if dfSurgeon.iloc[s][1] == 1:
    #                         if dfSecond.iloc[a][1] == 1:
    #                             for t in range(nSlot//T):
    #                                 SD[(s,a,t%2,d%5)] = 1;
    #                     if dfSurgeon.iloc[s][2] == 1:
    #                         if dfSecond.iloc[a][2] == 1:
    #                             for t in range(nSlot//T, nSlot):
    #                                 SD[(s,a,t%2,d%5)] = 1;

    # Disponibilidad de paciente
    DISP = np.ones(nPatients);
    
    def busquedaType(especialidad):
        indice = 0;
        for i in range(len(process)):
            if (especialidad == dfType.iloc[i][0]):
                indice = i;
        return indice
    
    # Compatibilidad paciente-cirujano
    SP = np.zeros((nPatients, nSurgeons), dtype=int);
    contador = 0;
    for p in patient:
        #print(dfPatient.iloc[p][21])
        for s in surgeon:
            if busquedaType(dfPatient.iloc[p]["especialidad"]) == busquedaType(dfSurgeon.iloc[s][9]):
                #print(dfPatient.iloc[p][21])
                SP[p][s] = 1;            

    # Diccionario de paciente
    dic_p = {p: [0, 0, 0, 0, 0] for p in patient};
    for p in patient:
        #dic_p[p][0] = list_patient[p] #paciente y número aleatorio asociado (entre 0 y 1)
        dic_p[p][1] = busquedaType(dfPatient.iloc[p]["especialidad"]) # ID Especialidad
        dic_p[p][2] = dfPatient.iloc[p]["nombre"]; #Nombre del paciente
        dic_p[p][3] = p; # ID
        dic_p[p][4] = dfPatient.iloc[p]["especialidad"]; # Especialidad requerida

    # Compatibilidad quirófano-paciente
    AOR = np.zeros((nPatients, nRooms, nSlot, 5));
    dicOR = {o:[] for o in room};
    j = [];
    z = [];
    ns = 0;
    for o in room:
        if o == 0:
            for d in range(5):
                for e in range(2):
                    #print(e)
                    if e == 0:                    
                        for t in range(len(slot)//2):

                            j = dfdisORA[e][d%5];
                            j = j.split("#")
                            for a in range(len(j)):
                                z=j[a]
                                dicOR[ns]=[o,d%5,t,z]
                                ns+=1
                    if e==1:
                        #print('paso')
                        for t in range(int(len(slot)/2),len(slot)):

                            j=dfdisORA[e][d%5]
                            j=j.split("#")
                            for a in range(len(j)):
                                z=j[a]
                                dicOR[ns]=[o,d%5,t,z]
                                ns+=1
        if o==1:
            for d in range(5):
                for e in range(2):
                    if e==0:                    
                        for t in range(0,int(len(slot)/2)):
                            j=dfdisORB[e][d%5]
                            j=j.split("#")
                            for a in range(len(j)):
                                z=j[a]
                                dicOR[ns]=[o,d,t,z]
                                ns+=1
                    if e==1:
                        for t in range(int(len(slot)/2),len(slot)):
                            j=dfdisORB[e][d%5]
                            j=j.split("#")
                            for a in range(len(j)):
                                z=j[a]
                                dicOR[ns]=[o,d,t,z]
                                ns+=1
        if o==2:
            for d in range(5):
                for e in range(2):
                    if e==0:                    
                        for t in range(0,int(len(slot)/2)):
                            j=dfdisORC[e][d%5]
                            j=j.split("#")
                            for a in range(len(j)):
                                z=j[a]
                                dicOR[ns]=[o,d,t,z]
                                ns+=1
                    if e==1:
                        for t in range(int(len(slot)/2),len(slot)):
                            j=dfdisORC[e][d%5]
                            j=j.split("#")
                            for a in range(len(j)):
                                z=j[a]
                                dicOR[ns]=[o,d,t,z]
                                ns+=1
        if o==3:
            for d in range(5):
                for e in range(2):
                    if e==0:                    
                        for t in range(0,int(len(slot)/2)):
                            j=dfdisORD[e][d%5]
                            j=j.split("#")
                            for a in range(len(j)):
                                z=j[a]
                                dicOR[ns]=[o,d,t,z]
                                ns+=1
                    if e==1:
                        for t in range(int(len(slot)/2),len(slot)):
                            j=dfdisORD[e][d%5]
                            j=j.split("#")
                            for a in range(len(j)):
                                z=j[a]
                                dicOR[ns]=[o,d,t,z]
                                ns+=1

    p = 0;
    o = 0;
    t = 0;

    for ns in range(len(dic_p)):
        for nP in range(len(dicOR)):
            #print(dicOR[nP][3])
            if str(dic_p[ns][1])==dicOR[nP][3]:
                p = dic_p[ns][3];
                o = dicOR[nP][0];
                t = dicOR[nP][2];
                d = dicOR[nP][1];
                AOR[p][o][t][d%5] = 1;

    OT = np.zeros(nPatients, dtype=int);
    for p in patient:
        OT[p] = int(dfPatient.iloc[p]["tipo"]);
    #print("_" * 160);

    nFichas = int((parametroFichas * 4 * nSlot * len(room) * 2 * 3 )/(len(surgeon)**0.5));
    #print("Nuevas fichas:", nFichas);
    #print('Datos Obtenidos.');

    ###################################################################################################################################
    #########                                               Solución Inicial                                                  #########
    ###################################################################################################################################

    def compress(o, d, t):
        return o * nSlot * nDays + d * nSlot + t

    def decompress(val):
        o = (val) // (nSlot * nDays);
        temp = (val) % (nSlot * nDays);
        d = temp // nSlot;
        t = temp % nSlot;
        return o, d, t

    def WhichExtra(o,t,d,e):
        try:
            return int(extras[o][t][d%5][e]);
        except:
            print(f'extras: d:{d},t:{t},o:{o},e:{e}');
            print(stop)

    dictCosts = {};

    for s in surgeon:
        for a in second:
            for _ in range(nSlot * nDays * len(room)):
                o, d, t = decompress(_);
                dictCosts[(s, a, _)] = int(dfdisAffi.iloc[a][s+1] + dfdisAffiDiario.iloc[d%5][s+1] + sum(Ex[i][(s,WhichExtra(o,t//T,d,i)-1)] for i in range(num_ext)) + dfdisAffiBloque.iloc[t//T][s+1]);
    
    def generar_solucion_inicial(VERSION="A"):
        all_personnel = set(surgeon).union(second);
        timeUsedMap = { person: set() for person in all_personnel};
        boundary = nSlot//2;

        def encontrar_pacientes_cirujanos(p):
            compatibles = [];
            for s in surgeon:
                if SP[p][s] == 1:
                    for a in second:
                        if a != s and COIN[s][a] == 0:
                            compatibles.append((p, s, a, OT[p]));
            return compatibles

        def cirujano_disponible(s, a, o, d, t, duracion):
            for b in range(int(duracion)):
                if (d, t + b) in timeUsedMap[s]:
                    return False
                if (d, t + b) in timeUsedMap[a]:
                    return False
            return True

        def asignar_paciente(p, s, a, o, d, t, duracion):
            if asignP[p] == -1:
                asignP[p] = compress(o, d, t);
                for b in range(int(duracion)):
                    or_schedule[o][d][t + b] = p;
                    surgeon_schedule[s][d][t + b] = p;
                    surgeon_schedule[a][d][t + b] = p;

                    id_block = compress(o, d, t + b);
                    dictS[id_block] = s;
                    dictA[id_block] = a;
                    timeUsedMap[s].add((d, t + b));
                    timeUsedMap[a].add((d, t + b));
                asignS[s].add((o, d, t, duracion));
                asignA[a].add((o, d, t, duracion));

        patient_sorted = sorted(patient, key=lambda p: I[(p, 0)], reverse=True);

        asignP = [-1] * len(patient);
        asignS = {s: set() for s in surgeon};
        asignA = {a: set() for a in second};
        dictS  = {};
        dictA  = {};
        fichas = {(s, d): nFichas * (d+1) for s in surgeon for d in day};

        surgeon_schedule = {s: [[-1 for t in slot] for d in day] for s in surgeon};
        or_schedule = {o: [[-1 for t in slot] for d in day] for o in room};

        for p in patient_sorted:
            assigned = False
            duracion_p = OT[p]
            for o in room:
                for d in day:
                    for t in range(nSlot - duracion_p + 1):
                        if duracion_p > 1:
                            if t < boundary and (t + duracion_p) > boundary:
                                continue
                        if all(AOR[p][o][t + b][d % 5] == 1 for b in range(duracion_p)):
                            #if es_bloque_disponible(o, d, t, duracion_p):
                            if all(or_schedule[o][d][t + b] == -1 for b in range(duracion_p)):
                                resultados = encontrar_pacientes_cirujanos(p)
                                for (p_res, s, a, dur) in resultados:
                                    if cirujano_disponible(s, a, o, d, t, dur):
                                        if (dfdisAffi.iloc[a][s+1] >= level_affinity*(VERSION=="B") and
                                            dfdisAffiDiario.iloc[d % 5][s+1] >= level_affinity*(VERSION=="B") and
                                            dfdisAffiBloque.iloc[t // (nSlot // 2)][s+1] >= level_affinity*(VERSION=="B")):
                                            checks = 0;
                                            for i in range(num_ext):
                                                e = int(WhichExtra(o,t//8,d%5,i));
                                                if Ex[i][(s,e-1)] >= level_affinity*(VERSION=="B"):
                                                    checks += 1;
                                            if checks >= num_ext*(VERSION=="B"):
                                                cost = dictCosts[(s, a, compress(o, d, t))]
                                                if all(fichas[(s, d_aux)] >= cost*(VERSION=="C") for d_aux in range(d, len(day))):
                                                    asignar_paciente(p_res, s, a, o, d, t, dur)
                                                    for d_aux in range(d, len(day)):
                                                        fichas[(s, d_aux)] -= cost;
                                                        #print(f"{d_aux}. costo: {cost}, fichas {s} {d}:", fichas[(s,d)])
                                                    assigned = True
                                                    break
                                    if assigned:
                                        break
                    if assigned:
                        break
                if assigned:
                    break

        #print("Solución inicial creada...")
        return asignP, dictS, dictA

    ###################################################################################################################################
    #########                                           Contruccion modelo cplex                                              #########
    ###################################################################################################################################
    
    inicio_carga = time.time();
    import psutil
    from docplex.mp.model import Model
    import docplex.mp.solution as Solucion

    def WhichExtra(o,t,d,e):
        try:
            return int(extras[o][t][d%5][e]);
        except:
            print(f'extras: d:{d%5},t:{t},o:{o},e:{e}');
            #print(stop)

    #mdl=Model('MIP Model',ignore_names=True,checker='off');
    mdl = Model('MIP Model',checker='off');

    # Variables
    x = mdl.binary_var_dict(arcox, name='x');
    y = mdl.binary_var_dict(arcoy, name='y');
    z = mdl.binary_var_dict(arcoz, name='z');
    f = mdl.integer_var_dict(arcof, lb=0, name='f');
    ts = mdl.integer_var_dict(arcot, lb=0, ub=nSlot-1, name='ts');
    te = mdl.integer_var_dict(arcot, lb=0, ub=nSlot-1, name='te');

    def decompress(val):
        o = (val) // (nSlot * nDays);
        temp = (val) % (nSlot * nDays);
        d = temp // nSlot;
        t = temp % nSlot;
        return o, d, t
    
    fichas = nFichas;
    def multiplicador(dia):
        return nDays//(dia+1);
    
    print("Cargando función objetivo...")
    if version == "C":
        mdl.maximize(mdl.sum(1000*I[(p,d)]*y[(p,o,d)] for p in patient for o in room for d in day) 
                     - mdl.sum(f[(s,d)] * multiplicador(d) for s in surgeon for d in day));
    else:
        mdl.maximize(mdl.sum(1000*I[(p,d)]*y[(p,o,d)] for p in patient for o in room for d in day));       
    
    # Restricciones
    '''
    print("Cargando (2)...");
    for s in surgeon:
        for d in day:
            mdl.add_constraint(f[(s,d)] == fichas + f[(s,d-1)] - 
                               sum((dfdisAffi.iloc[a][s+1] + dfdisAffiDiario.iloc[d][s+1]
                                    + sum(Ex[i][(s,WhichExtra(o,t//4,d,i)-1)] for i in range(num_ext))
                                    + dfdisAffiBloque.iloc[t//4][s+1]) * (1/OT[p]) * x[(p,o,s,a,t,d)] 
                                   for p in patient for o in room for a in second for t in slot));

    for s in surgeon:
        mdl.add_constraint(f[(s,-1)] == 0);
    '''
    array = dfdisAffi.values
    lala = dfdisAffiDiario.values
    lele = dfdisAffiBloque.values
    
    if version == "B":
        print("Cargando (2-B)...");
        for p in patient:
            for s in surgeon:
                for a in second:
                     mdl.add_constraint(z[(p,s,a)]*dfdisAffi.iloc[a][s+1] >= level_affinity*z[(p,s,a)]);

        for p in patient:
            for s in surgeon:
                for t in slot:
                    for d in day:
                        mdl.add_constraint(mdl.scal_prod(mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for a in second),
                                                         dfdisAffiDiario.iloc[d%5][s+1]) >= level_affinity*mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for a in second));

        for p in patient:
            for s in surgeon:
                for t in slot:
                    mdl.add_constraint(mdl.scal_prod(mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for a in second for d in day),
                                                     dfdisAffiBloque.iloc[t//(nSlot//2)][s+1]) >= level_affinity*mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for a in second for d in day));
    
        for i in range(num_ext):
            for o in room:
                for s in surgeon:
                    for t in slot:
                        for d in day:
                            e = int(WhichExtra(o,t//8,d%5,i));
                            try:
                                mdl.add_constraint(mdl.scal_prod(mdl.sum_vars(x[(p,o,s,a,t,d)] for p in patient for a in second),Ex[i][(s,e-1)]) >= level_affinity*mdl.sum_vars(x[(p,o,s,a,t,d)] for p in patient for a in second));
                            except:
                                print(f'i:{i}, o:{o}, s:{s},t:{t},d:{d},e:{e}');
    

    elif version == "C":

        dictCosts = {};
        for s in surgeon:
            for a in second:
                for _ in range(nSlot * nDays * len(room)):
                    o, d, t = decompress(_);
                    dictCosts[(s, a, _)] = int(dfdisAffi.iloc[a][s+1] + dfdisAffiDiario.iloc[d%5][s+1] + sum(Ex[i][(s,WhichExtra(o,t//T,d,i)-1)] for i in range(num_ext)) + dfdisAffiBloque.iloc[t//T][s+1]);
        '''        
        print("Cargando (2-C)...");
        for s in surgeon:
            for d in day:
                mdl.add_constraint(f[(s,d)] == fichas + f[(s,d-1)] - 
                                   #mdl.sum((dfdisAffi.iloc[a][s+1] + dfdisAffiDiario.iloc[d][s+1]
                                   mdl.sum((array[a][s] + lala[d%5][s]
                                       + mdl.sum(Ex[i][(s,WhichExtra(o,t//T,d,i)-1)] for i in range(num_ext))
                                   #    + dfdisAffiBloque.iloc[t//4][s+1]) * (1/OT[p]) * x[(p,o,s,a,t,d)]
                                       + lele[t//T][s]) * (1/OT[p]) * x[(p,o,s,a,t,d)]  
                                       for p in patient for o in room for a in second for t in slot));
        '''
        for s in surgeon:
            mdl.add_constraint(f[(s,-1)] == 0);
            for d in day:
                mdl.add_constraint(f[(s,d)] == fichas + f[(s,d-1)] - mdl.sum(dictCosts[(s, a, compress(o, d, t))] * (1/OT[p]) * x[(p,o,s,a,t,d)] 
                                                                               for p in patient for o in room for a in second for t in slot));
        
    print("Cargando (3)...");
    for p in patient:
        for t in slot:
            mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for s in surgeon for a in second for d in day) <= 1);
    
    
    print("Cargando (4)...");
    for o in room:
        for t in slot:
            for d in day:
                mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,t,d)] for p in patient for s in surgeon for a in second) <= 1);
    
    print("Cargando (5)...");
    for s in surgeon:
        for t in slot:
            for d in day:
                mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,t,d)] for p in patient for o in room for a in second) <= 1);
    
    print("Cargando (6)...");
    for a in second:
        for t in slot:
            for d in day:
                mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,t,d)] for p in patient for o in room for s in surgeon) <= 1);

    
    print("Cargando (7)...");
    for p in patient:
        for s in surgeon:
            for a in second:
                for t in slot:
                    for d in day:
                        mdl.add_constraint(mdl.sum((mdl.scal_prod((x[(p,o,s,a,t,d)] for o in room), COIN[(s,a)]))) <= 0);
                        
    print("Cargando (8)...");
    for p in patient:
        mdl.add_constraint(mdl.sum_vars(y[(p,o,d)] for o in room for d in day) <= 1);
    
    print("Cargando (9)...");
    for p in patient:
        mdl.add_constraint(mdl.sum_vars(z[(p,s,a)] for s in surgeon for a in second) <= 1);
     
       
    #print("Cargando (10)...");
    #(4.10)
    #for p in patient:
    #    for o in room:
    #        for d in day:
    #            mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,t,d)] for s in surgeon for a in second for t in slot)
    #                               <=len(slot)*y[(p,o,d)])         

    
    print("Cargando (11)...");
    #(4.11*)
    for p in patient:
         mdl.add_constraint(mdl.sum_vars(y[(p,o,d)] for o in room for d in day) <= mdl.sum_vars(z[(p,s,a)] for s in surgeon for a in second));
    
    
    # print("Cargando (12)...");
    # for p in patient:
    #     for s in surgeon:
    #         for a in second:
    #             for t in slot:
    #                 for d in day:
    #                     mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room) <= SD[(s,a,t,d%5)] * z[(p,s,a)]);
    
    
    print("Cargando (13)...");
    for p in patient:
        mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for s in surgeon for a in second for t in slot for d in day) <= nSlot * DISP[p]);
        
    print("Cargando (14)...");
    for p in patient:
        for s in surgeon:
            for d in day:
                mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for a in second for t in slot) <= nSlot * SP[(p,s)]);

    print("Cargando (15)...");
    for p in patient:
        for o in room:
            for t in slot:
                for d in day:
                    mdl.add_constraint(mdl.sum(x[(p,o,s,a,t,d)] for s in surgeon for a in second) <= AOR[(p,o,t,d%5)]);
    
    
    print("Cargando (16)...");
    for p in patient:
        for s in surgeon:
            for a in second:
                mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for t in slot for d in day) >= (OT[p]) * z[(p,s,a)]);

    print("Cargando (17)...");
    for t in slot:
        for p in patient:
            mdl.add_constraint(ts[(p)] <= t * mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for s in surgeon for a in second for d in day)
                              + nSlot * (1 - mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for s in surgeon for a in second for d in day)));
    
    print("Cargando (18)...");
    for t in slot:
        for p in patient:
            mdl.add_constraint(te[(p)] >= t * mdl.sum_vars(x[(p,o,s,a,t,d)] for o in room for s in surgeon for a in second for d in day));
    
    print("Cargando (19)...");
    for p in patient:
        mdl.add_constraint(ts[(p)] == te[(p)] - OT[p] + 1);
        
    print("Cargando (20)..."); #No se puede usar un bloque de la mañana y de la tarde para una misma operación
    for p in patient:
        for d in day:
            mdl.add_constraint(mdl.sum_vars(x[(p,o,s,a,T-1,d)] for o in room for s in surgeon for a in second) <=
                              1 - mdl.sum_vars(x[(p,o,s,a,T,d)] for o in room for s in surgeon for a in second));
           
    print("Cargando (21)..."); #Evitar asignar a rooms diferentes cuando la operación dura más de un bloque
    for p in patient:
        for d in day:
            for o in room:
                mdl.add_constraint(mdl.sum(x[(p, o, s, a, t, d)] for t in slot for s in surgeon for a in second) <= OT[p] * y[(p, o, d)]);

    for p in patient:
        mdl.add_constraint(mdl.sum(y[(p, o, d)] for o in room for d in day) <= 1);
    
    
    fin_carga = time.time();
    tiempo_carga = str(np.around(fin_carga - inicio_carga, decimals=2));
    #print(mdl.export_to_string())
    print('Restricciones Cargadas.\nComenzando resolución...\n');
    time_inicio = time.time();
    mdl.parameters.timelimit = time_limit;
    mdl.parameters.threads = 1;

    ########### Solución Inicial

    def validar_solucion_inicial(mdl, warmstart):
        for variable, valor in warmstart.iter_var_values():
            if valor > 0:
                print(f"Comprobando variable: {variable} con valor: {valor}")
                try:
                    mdl.add_constraint(variable == valor, f"check_{variable}")
                    solucion_temporal = mdl.solve()
                    if mdl.get_solve_status().name == 'INFEASIBLE_SOLUTION':
                        print(f"Variable {variable} con valor {valor} viola las restricciones.")
                    #mdl.remove_constraint(f"check_{variable}")
                except Exception as e:
                    print(f"Error al validar la variable {variable}: {e}")

    if solucion_inicial == True:
        ini = generar_solucion_inicial(VERSION=version);
        fichas = [[nFichas * (d+1) for d in range(len(day))] for s in surgeon];
        warmstart = mdl.new_solution();
        print("Pacientes asignados en la solución inicial:", len([item for item in ini[0] if item >= 0]));
        for p in range(len(ini[0])):
            if ini[0][p] >= 0:
                s = ini[1][ini[0][p]];
                a = ini[2][ini[0][p]];
                o, d, t = decompress(ini[0][p]);
                warmstart.add_var_value(x[(p,o,s,a,t,d)], 1);
                warmstart.add_var_value(y[(p,o,d)], 1);
                warmstart.add_var_value(z[(p,s,a)], 1);
                warmstart.add_var_value(ts[(p)], t);
                warmstart.add_var_value(te[(p)], t + int(OT[p]) - 1);
                cost  = dictCosts[(s, a, ini[0][p])];
                for d_aux in range(d, nDays):
                    fichas[s][d_aux] -= cost;
        if version == "C":
            for s in surgeon:
                for d in day:
                    warmstart.add_var_value(f[(s,d)], fichas[s][d]);

        #validar_solucion_inicial(mdl, warmstart);
        mdl.add_mip_start(warmstart);

    
    solucion = mdl.solve(log_output=True);
    
    memoria_usada = np.around(psutil.Process().memory_info().rss / 10**6, decimals=2);
    print("Memoria usada:", memoria_usada, "megabytes");
    
    #list_sol[14] = str(memoria_usada) + "\n";
    dict_sol["Memoria"] = [memoria_usada];

    #print('status:',mdl.get_solve_status());
    if str(mdl.get_solve_status()) == "JobSolveStatus.OPTIMAL_SOLUTION":
        #list_sol[5] = "Opt";
        dict_sol["Status"] = ["Opt"];
    else:
        #list_sol[5] = "Fact";
        dict_sol["Status"] = ["Fact"];
        
    if mdl.get_solve_status().name == 'INFEASIBLE_SOLUTION':
        dict_sol["Status"] = ["Infact"];
        dict_sol["Valor FO"] = [0];
        dict_sol["Best Bound"] = [0];
        dict_sol["Rel GAP"] = [0];
        time_fin = time.time()
        time_ejecucion = time_fin - time_inicio
        print("Tiempo ejecucion:",time_ejecucion);
        dict_sol["Tiempo Carga"] = [tiempo_carga];
        dict_sol["Tiempo Ejec"] = [np.around(time_ejecucion, decimals=2)];
        dict_sol["Pacientes Atend"] = [0];
        dict_sol["Prioridad"] = [0];
        dict_sol["Avg Fichas"] = [0];
        dict_sol["Std Fichas"] = [0];
        dict_sol["Avg Cirug"] = [0];
        dict_sol["Std Cirug"] = [0];
        dict_sol["Avg Ratio"] = [0];
        dict_sol["Std Ratio"] = [0];
        dict_sol["Ocupación"] = [0];
        dfAux = pd.DataFrame(dict_sol);
        dfSolucion = pd.concat([dfSolucion, dfAux]);
        dfSolucion.to_excel(f"../output/LPM/{entrada}_output.xlsx", index=False);
        continue;

    try:
        solucion.display();
    except:
        dfAux = pd.DataFrame(dict_sol);
        dfSolucion = pd.concat([dfSolucion, dfAux]);
        dfSolucion.to_excel(f"../output/LPM/{entrada}_output.xlsx", index=False);
        print("No hay solución.");
        continue;

    print('1:',solucion.get_objective_value())
    #list_sol[1] = str(solucion.get_objective_value());
    dict_sol["Valor FO"] = [solucion.get_objective_value()];
    
    print('Best bound:',solucion.solve_details.best_bound)
    #list_sol[2] = str(round(solucion.solve_details.best_bound,2));
    dict_sol["Best Bound"] = [round(solucion.solve_details.best_bound, 2)];
    
    print('Rgap:',solucion.solve_details.mip_relative_gap)
    #list_sol[6] = str(round(solucion.solve_details.mip_relative_gap*100,2));
    dict_sol["Rel GAP"] = [round(solucion.solve_details.mip_relative_gap, 2)];
    
    #print('Time:',solucion.solve_details.time)

    time_fin = time.time();
    time_ejecucion = time_fin - time_inicio;
    #print("Tiempo ejecucion:",time_ejecucion)
    
    #list_sol[3] = str(tiempo_carga);
    dict_sol["Tiempo Carga"] = [tiempo_carga];
    
    #list_sol[4] = str(np.around(time_ejecucion, decimals=2));
    dict_sol["Tiempo Ejec"] = [round(time_ejecucion, 2)];

    pacientes_atend = [];
    fichas_ocup = [0 for i in range(len(surgeon))];
    cirujanos_atend_num = [0 for i in range(len(surgeon))];
    prioridad = 0;
    ocupacion = 0;
    ratios = [0 for i in range(len(surgeon))];
    PROG = {e:[None, None, None, None, None, None, None, None, None, None] for e in range(nSlot*nDays*len(room) + nDays*(len(room) + 1) + 2)};
    
    c = 0;
    for d in day: 
        PROG[c][0] = 'DIA ' + str(d);
        c += 1;
        for o in room: 
            PROG[c][0] = 'PABELLON ' + str(o);
            c += 1;
            for t in slot:
                for p in patient:
                    for s in surgeon:
                        #if version == "C":
                        #    fichas_ocup[s] = nFichas*nDays - f[(s,nDays-1)].solution_value;
                        for a in second:
                            if x[(p,o,s,a,t,d)].solution_value == 1:
                                PROG[c][0] = 'BLOQUE ' + str(t) 
                                PROG[c][1] = dfPatient.iloc[p]["nombre"];
                                if dfPatient.iloc[p]["id"] not in pacientes_atend:
                                    pacientes_atend.append(dfPatient.iloc[p]["id"]);
                                    fichas_ocup[s] += dictCosts[s, a, compress(o, d, t)];
                                    cirujanos_atend_num[s] += 1;
                                    prioridad += I[(p,d)];
                                    ocupacion += int(OT[p]);
                                PROG[c][2] = dfSurgeon.iloc[s][0];
                                PROG[c][3] = dfSecond.iloc[a][0];
                                PROG[c][4] = dfPatient.iloc[p]["especialidad"];
                                #PROG[c][5] = f[(s,d)].solution_value;
                                PROG[c][5] = dfdisAffi.iloc[a][s+1];
                                aux = 0;
                                for i in range(num_ext):
                                    aux += int(Ex[i][(s,WhichExtra(o,t//T,d,i)-1)]);
                                PROG[c][6] = aux;
                                PROG[c][7] = dfdisAffiDiario.iloc[d%5][s+1];
                                PROG[c][8] = dfdisAffiBloque.iloc[t//T][s+1];
                                PROG[c][9] = int(PROG[c][5]) + int(PROG[c][6]) + int(PROG[c][7]) + int(PROG[c][8]);
                                c += 1;         
    def export_results():
        pac = [];
        cir = [];
        sec = [];
        hab = [];
        dia = [];
        blo = [];
        for p in patient:
            for o in room:
                for s in surgeon:
                    for a in second:
                        for t in slot:
                            for d in day:
                                if x[(p,o,s,a,t,d)].solution_value == 1 and p not in pac:
                                    pac.append(p);
                                    cir.append(s);
                                    sec.append(a);
                                    hab.append(o);
                                    dia.append(d);
                                    blo.append(t);
        return pac, cir, sec, hab, dia, blo;
    
    print(export_results());
    
    dfPROG = pd.DataFrame({'BLOQUE': [PROG[c][0] for c in PROG],
        'PACIENTE': [PROG[c][1] for c in PROG],
        '1ER CIRUJANO': [PROG[c][2] for c in PROG],
        '2DO CIRUJANO': [PROG[c][3] for c in PROG],
        'TIPO PROC': [PROG[c][4] for c in PROG],
        'FICHAS CIR': [PROG[c][5] for c in PROG],
        'FICHAS EXT': [PROG[c][6] for c in PROG],
        'FICHAS DIA': [PROG[c][7] for c in PROG],
        'FICHAS BLO': [PROG[c][8] for c in PROG],
        'TOTAL FICHAS': [PROG[c][9] for c in PROG]});
    
    #dfSTATS = pd.DataFrame([[dfSurgeon.iloc[i][0] + ": ", str(f[(i,nDays-1)].solution_value)] for i in surgeon]);
    #print(dfSTATS);

    #writer = ExcelWriter(carpeta + "Resultados/Linear Programming Model/LPM C/" + "LPM_C_"+instancia[INS][0]+
    #                     "_"+instancia[INS][1]+"_"+"x"+"_"+instancia[INS][3]+"_"+instancia[INS][4]+".xlsx");
    
    dfPROG.to_excel(f"../output/LPM/{entrada}_Schedule_v{version}p{patient_code}n{nPatients}s{nSurgeons}d{nDays}.xlsx", index=False);
    #dfSTATS.to_excel(f"./Output/Stats_v{version}p{patient_code}s{nSurgeons}d{nDays}.xlsx", index=False);
    
    print("Pacientes atendidos:", len(pacientes_atend));
    
    #list_sol[7] = str(len(pacientes_atend));
    dict_sol["Pacientes Atend"] = [len(pacientes_atend)];

    dict_sol["Prioridad"] = [prioridad];
    
    #list_sol[8] = str(round(np.mean(fichas_ocup),1));
    print(fichas_ocup)
    dict_sol["Avg Fichas"] = [round(np.mean([i for i in fichas_ocup if i != 0]), 2)];
    
    #list_sol[9] = str(round(np.std(fichas_ocup),1));
    dict_sol["Std Fichas"] = [round(np.std([i for i in fichas_ocup if i != 0]), 2)];
    
    print(cirujanos_atend_num)
    #list_sol[10] = str(round(np.mean(cirujanos_atend_num),1));
    dict_sol["Avg Cirug"] = [round(np.mean([i for i in cirujanos_atend_num if i != 0]), 2)];
    
    #list_sol[11] = str(round(np.std(cirujanos_atend_num),1));
    dict_sol["Std Cirug"] = [round(np.std([i for i in cirujanos_atend_num if i != 0]), 2)];
    
    for i in range(nSurgeons):
        if cirujanos_atend_num[i] != 0:
            ratios[i] = round(fichas_ocup[i]/cirujanos_atend_num[i], 2);
            
    #list_sol[12] = str(round(np.mean(ratios),1));
    dict_sol["Avg Ratio"] = [round(np.mean([i for i in ratios if i != 0]), 2)];
    
    #list_sol[13] = str(round(np.std(ratios),1)) + "\n";
    dict_sol["Std Ratio"] = [round(np.std([i for i in ratios if i != 0]), 2)];

    dict_sol["Ocupación"] = [round(ocupacion/(nSlot*nDays*len(room)), 2)];
    
    dfAux = pd.DataFrame(dict_sol);
    dfSolucion = pd.concat([dfSolucion, dfAux]);
            
    dfSolucion.to_excel(f"../output/LPM/{entrada}_output.xlsx", index=False);
print('Fin del programa.');

Versión: C;
Pacientes: low; Número de Pacientes: 150;
Días: 5;
Afinidad: 1; Cirujanos: 24;
Fichas: 40.
Cargando función objetivo...
Cargando (3)...
Cargando (4)...
Cargando (5)...
Cargando (6)...
Cargando (7)...
Cargando (8)...
Cargando (9)...
Cargando (11)...
Cargando (13)...
Cargando (14)...
Cargando (15)...
Cargando (16)...
Cargando (17)...
Cargando (18)...
Cargando (19)...
Cargando (20)...
Cargando (21)...
Restricciones Cargadas.
Comenzando resolución...

Pacientes asignados en la solución inicial: 141
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 130360.0000.
Presolve has eliminated 6624174 rows and 24 columns...
Presolve has improved bounds 188 times...
Presolve has eliminated 7056436 rows and 26907832 columns...
Presolve has i

: 

In [24]:
dict_sol

{'Instancia': ['vCp1n200s12d3'],
 'Memoria': [np.float64(22183.71)],
 'Status': ['Fact'],
 'Valor FO': [228741.0],
 'Best Bound': [230801.56],
 'Rel GAP': [0.01],
 'Tiempo Carga': ['91.41'],
 'Tiempo Ejec': [1801.84],
 'Pacientes Atend': [95],
 'Prioridad': [np.int64(230)],
 'Avg Fichas': [np.float64(76.2)],
 'Std Fichas': [np.float64(36.5)],
 'Avg Cirug': [np.float64(7.9)],
 'Std Cirug': [np.float64(4.2)],
 'Avg Ratio': [np.float64(10.6)],
 'Std Ratio': [np.float64(2.5)],
 'Ocupación': [0.6]}

In [3]:
def compress(o, d, t):
    return o * nSlot * nDays + d * nSlot + t

def decompress(val):
    o = (val) // (nSlot * nDays);
    temp = (val) % (nSlot * nDays);
    d = temp // nSlot;
    t = temp % nSlot;
    return o, d, t

pacientes = [-1 for p in patient];
primarios = {};
secundarios = {};

for p in patient:
    for o in room:
        for s in surgeon:
            for a in second:
                for d in day:
                    for t in slot:
                        if x[(p,o,s,a,t,d)].solution_value==1:
                            bloque = compress(o, d, t);
                            if pacientes[p] == -1:
                                pacientes[p] = bloque;
                            primarios[bloque] = s;
                            secundarios[bloque] = a;
            
print(pacientes, primarios, secundarios)
print([int(OT[p]) for p in patient])

[-1, 111, 73, 98, 30, 106, -1, -1, -1, 39, 94, 107, -1, 103, -1, 119, 109, 19, 114, 51, 112, 86, 99, 87, -1, -1, 46, 124, 50, 45, 77, 17, 71, 69, -1, 13, 48, 15, 11, -1, -1, 40, 31, 80, 126, 42, 64, -1, -1, -1, 127, 43, 52, 85, -1, 66, 63, 1, -1, 89, 102, -1, -1, -1, 60, -1, 117, 22, 88, 59, 28, 75, 21, -1, 29, -1, -1, -1, -1, 26, 6, 110, 54, -1, -1, 121, -1, 27, 14, 68, 74, -1, -1, 67, 16, -1, -1, 18, 20, 118, 104, 55, 9, 90, -1, 82, 24, -1, 96, 12, 92, 108, -1, -1, 0, 113, -1, 116, 25, -1, -1, -1, -1, 4, 56, 91, 61, 2, 53, 65, -1, 72, -1, 62, 101, 123, 79, 8, -1, 10, 70, 44, -1, 57, 58, -1, 100, 49, -1, 105, -1, 81, 3, 84, 5, -1, -1, 78, -1, 41] {111: 1, 73: 5, 98: 2, 30: 4, 106: 1, 39: 0, 94: 0, 95: 0, 107: 1, 103: 2, 119: 2, 109: 1, 19: 4, 114: 2, 115: 2, 51: 3, 112: 2, 86: 0, 99: 2, 87: 0, 46: 0, 124: 2, 125: 2, 50: 3, 45: 0, 77: 5, 17: 4, 71: 5, 69: 5, 13: 4, 48: 3, 15: 4, 11: 4, 40: 0, 31: 4, 80: 0, 126: 2, 42: 0, 64: 5, 127: 2, 43: 0, 52: 3, 85: 0, 66: 5, 63: 3, 1: 0, 89: 0, 10